In [ ]:
# run for first time setup
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension
# if the dropdown lists don't appear, close browser window and restart jupyter notebook

In [ ]:
import login
login.connect()
import schema.common_mice, schema.common_exp
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import numpy as np

Navigate the Mouse and Sacrificed tables

In [ ]:
# create widgets
user_widget = widgets.Dropdown(options = [""] + list(schema.common_mice.Investigator().fetch("username")))
status_widget = widgets.Dropdown(options = ["", "alive", "sacrificed"])
batch_widget = widgets.Dropdown(options = [""] + list(np.unique(schema.common_mice.Mouse().fetch("batch"))), value="")
strain_widget = widgets.Dropdown(options = [""] + list(np.unique(schema.common_mice.Mouse().fetch("strain"))), value="")
sex_widget = widgets.Dropdown(options = ["", "M", "F", "U"] , value="")

# restrict batch and strain options to match selected user
def user_update(*args):
    query = schema.common_mice.Mouse()
    if user_widget.value != "":
        query = query & "username = '{}'".format(user_widget.value)
    batch_widget.options = [""] + list(np.unique(query.fetch("batch")))
    strain_widget.options = [""] + list(np.unique(query.fetch("strain")))
user_widget.observe(user_update, "value")

# show mice, restricting by the current widget selections
def show_mice(username, status, batch, strain, sex):
    query = schema.common_mice.Mouse()
    if username != "":
        query = schema.common_mice.Mouse() & "username = '{}'".format(username)
    if status == "alive":
        query = query - schema.common_mice.Sacrificed()
    elif status == "sacrificed":
        query = query * schema.common_mice.Sacrificed()
    if batch != "":
        query = query & "batch = '{}'".format(batch)
    if strain != "":
        query = query & "strain = '{}'".format(strain)
    if sex != "":
        query = query & "sex = '{}'".format(sex)
    return query

# make interactive plot
interact(show_mice, username=user_widget, status=status_widget, batch = batch_widget, strain = strain_widget, sex = sex_widget)

For a given mouse, show surgeries and injections

In [ ]:
# create widgets
user_widget = widgets.Dropdown(options = [""] + list(schema.common_mice.Investigator().fetch("username")))
mouse_widget = widgets.Dropdown(options = [""])
batch_widget = widgets.Dropdown(options = [""])

# restrict batch and mouse_id options to match selected user
def user_update(*args):
    query = schema.common_mice.Mouse()
    if user_widget.value != "":
        query = query & "username = '{}'".format(user_widget.value)
        mouse_widget.options = [""] + list(np.unique(query.fetch("mouse_id")))
        batch_widget.options = [""] + list(np.unique(query.fetch("batch")))
    else:
        mouse_widget.options = [""]
        batch_widget.options = [""]
user_widget.observe(user_update, "value")

# restrict mouse_id options to match selected batch
def batch_update(*args):
    query = schema.common_mice.Mouse() & "username = '{}'".format(user_widget.value)
    if batch_widget.value != "":
        query = query & "batch='{}'".format(batch_widget.value) 
        mouse_widget.options = [""] + list(np.unique(query.fetch("mouse_id")))
    else:
        mouse_widget.options = [""] + list(np.unique(query.fetch("mouse_id")))
batch_widget.observe(batch_update, "value")

def show_surgeries(username, mouse, batch):
    query_surgery = schema.common_mice.Surgery()
    if (username != ""):
        query_surgery = query_surgery & "username = '{}'".format(username)
    if (mouse != ""):
        query_surgery = query_surgery & "mouse_id = '{}'".format(mouse)
    display(query_surgery)
    display(schema.common_mice.Injection() & query_surgery)
    return

interact(show_surgeries, username=user_widget, mouse=mouse_widget, batch = batch_widget)